In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## PUNTO 1 REVISAR NUMERO DE COLUMNAS Y TIPO DE DATOS

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df.shape

(3066766, 19)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## PUNTO 2 CALCULAR LA COLUMNA DURATION

In [6]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [7]:
df['duration'] = df['duration'].dt.total_seconds() / 60

In [8]:
df.shape

(3066766, 20)

## PUNTO 3 REVISION Y ELIMINACION DE OUTLINERS

In [9]:
df.duration.std()

42.594351241920904

In [10]:
fraction_left = ((df.duration >= 1) & (df.duration <= 60)).mean()
fraction_left

0.9812202822125979

In [11]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
df.shape

(3009173, 20)

In [12]:
df.duration.std()

9.939385620145579

## PUNTO 4 One hot encoding

In [13]:
# Seleccionamos las columnas de interés
categorical = ['PULocationID', 'DOLocationID']

# Convertimos a string para asegurar que sean tratadas como categorias
df[categorical] = df[categorical].astype(str)

In [15]:
train_dicts = df[categorical].to_dict(orient='records')

In [16]:
# Inicializamos. sparse=True es el default, ahorra mucha memoria.
dv = DictVectorizer()

# Aprendemos y transformamos en un solo paso
X_train = dv.fit_transform(train_dicts)

In [17]:
# X_train.shape te devuelve (n_filas, n_columnas)
print(X_train.shape)

# Para ver solo las columnas (la respuesta a tu tarea):
print(X_train.shape[1])

(3009173, 515)
515


## PUNTO 5 ENTRENAMIENTO REGRESION LINEAL

In [18]:
# 1. Definir la variable objetivo (asegúrate de que 'df' es el mismo que usaste para X_train)
y_train = df['duration'].values

In [19]:
# 2. Entrenar el modelo
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [20]:
# 3. Hacer predicciones sobre el mismo conjunto de entrenamiento
y_pred = lr.predict(X_train)

# 4. Calcular el RMSE
# mean_squared_error calcula el promedio de los errores al cuadrado.
# squared=False nos da la raíz cuadrada directamente (RMSE).
rmse = mean_squared_error(y_train, y_pred, squared=False)

print(f"RMSE on train: {rmse}")

RMSE on train: 7.649261027919939


## PUNTO 6 EVALUACION DEL MODELO

In [21]:
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [22]:
df2['duration'] = df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime'] #CALCULAR DURATION PUNTO 2
df2['duration'] = df2['duration'].dt.total_seconds() / 60                    #CALCULAR DURATION EN MIN PUNTO 2
df2 = df2[(df2.duration >= 1) & (df2.duration <= 60)].copy()                 #ELIMINAR OUTLINERS  PUNTO 3

# Seleccionamos las columnas de interés
categorical = ['PULocationID', 'DOLocationID']                               #ONE HOT ENCODING PUNTO 4
# Convertimos a string para asegurar que sean tratadas como categorias
df2[categorical] = df2[categorical].astype(str)                              #ONE HOT ENCODING PUNTO 4


val_dicts = df2[categorical].to_dict(orient='records')                       #ONE HOT ENCODING PUNTO 4


# Transformar usando el vectorizador YA ENTRENADO (solo transform)
X_val = dv.transform(val_dicts)                                              #ONE HOT ENCODING PUNTO 4


In [23]:
# Definir la variable objetivo real de validación
y_val = df2['duration'].values

# Predecir con el modelo YA ENTRENADO
y_pred_val = lr.predict(X_val)

# Calcular RMSE en validación
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

print(f"RMSE on validation: {rmse_val}")

RMSE on validation: 7.811832638273232
